In [5]:
import numpy as np
import plotly.express as px

def motion_model(initial_state, increment, noise):
    xt, yt, zt, x_dot_t, y_dot_t, z_dot_t = initial_state
    delta_x_dot_t, delta_y_dot_t, delta_z_dot_t = increment
    noise_rx, noise_ry, noise_rz, noise_rx_dot, noise_ry_dot, noise_rz_dot = noise

    xt_new = xt + x_dot_t + delta_x_dot_t + noise_rx
    yt_new = yt + y_dot_t + delta_y_dot_t + noise_ry
    zt_new = zt + z_dot_t + delta_z_dot_t + noise_rz
    x_dot_t_new = x_dot_t + delta_x_dot_t + noise_rx_dot
    y_dot_t_new = y_dot_t + delta_y_dot_t + noise_ry_dot
    z_dot_t_new = z_dot_t + delta_z_dot_t + noise_rz_dot

    return [xt_new, yt_new, zt_new, x_dot_t_new, y_dot_t_new, z_dot_t_new]

def kalman_filter(previous_belief, increment, measurement):
    # State Transition Matrix
    A = np.array([[1, 0, 0, 1, 0, 0],
                    [0, 1, 0, 0, 1, 0],
                    [0, 0, 1, 0, 0, 1],
                    [0, 0, 0, 1, 0, 0],
                    [0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 1]])

    # Control Input Matrix
    B = np.array([[1, 0, 0],
                    [0, 1, 0],
                    [0, 0, 1],
                    [1, 0, 0],
                    [0, 1, 0],
                    [0, 0, 1]])

    # Measurement Matrix
    C = np.array([[1, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 0, 0],
                    [0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0],
                    [0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 1]])

    # DISCREPANCY
    # Motion Model Noise Noise Covariance Matrix
    
    sigma_rx, sigma_ry, sigma_rz = 1.2, 1.2, 1.2
    sigma_rx_dot, sigma_ry_dot, sigma_rz_dot = 0.01, 0.01, 0.01
    R = np.diag([sigma_rx**2, sigma_ry**2, sigma_rz**2, sigma_rx_dot**2, sigma_ry_dot**2, sigma_rz_dot**2])
    
    sigma_s = 7
    # Observation Model Noise Covariance Matrix
    Q = np.diag([sigma_s**2, sigma_s**2, sigma_s**2, sigma_s**2, sigma_s**2, sigma_s**2])

    # Initialize state estimate and error covariance estimate
    x_t = previous_belief[0]
    P_t = previous_belief[1]

    # Predict step
    # motion model
    x_t_pred = A.dot(x_t) + B.dot(increment)
    P_t_pred = A.dot(P_t).dot(A.T) + R

    # Update step
    # observation model
    y_t = measurement - C.dot(x_t_pred)
    S_t = C.dot(P_t_pred).dot(C.T) + Q
    K_t = P_t_pred.dot(C.T).dot(np.linalg.inv(S_t))

    x_t = x_t_pred + K_t.dot(y_t)
    P_t = (np.eye(6) - K_t.dot(C)).dot(P_t_pred)

    return x_t,P_t

T = 500
dt = 1
mean_belief = np.array([0, 0, 0, 0, 0, 0])
sigma = 10**-6
cov_belief = np.diag([sigma, sigma, sigma, sigma, sigma, sigma])

def control_policy(t):
    return [np.sin(t), np.cos(t), np.sin(t)]

observed_list = []
estimated_list = []

for t in range(T):
    increment = control_policy(t)
    # noise of motion model
    sigma_rx, sigma_ry, sigma_rz = 1.2, 1.2, 1.2
    sigma_rx_dot, sigma_ry_dot, sigma_rz_dot = 0.01, 0.01, 0.01
    R = np.diag([sigma_rx**2, sigma_ry**2, sigma_rz**2, sigma_rx_dot**2, sigma_ry_dot**2, sigma_rz_dot**2])
    noise = np.random.multivariate_normal([0, 0, 0, 0, 0, 0], R)
    # noise
    measurement = motion_model(mean_belief, increment,noise)
    observed_list.append(measurement)
    mean_belief,cov_belief = kalman_filter([mean_belief,cov_belief], increment, measurement)
    estimated_list.append(mean_belief)



In [16]:
observed_list = np.array(observed_list)[:,:3]
estimated_list = np.array(estimated_list)[:,:3]
combinedPos =  list(observed_list) + list(estimated_list)
true_measured =   ['Observed']*len(observed_list) + ['Estimated']*len(estimated_list)
dataFrame = {'x_column' : [v[0] for v in combinedPos], 'y_column' : [v[1] for v in combinedPos], 'z_column' : [v[2] for v in combinedPos], 'type' : true_measured}
fig = px.line_3d(dataFrame, x='x_column', y='y_column', z='z_column', color='type')
fig.show()

In [14]:
observed_list.shape

(500, 3)